# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 12 2023 2:52PM,254949,10,SNAP34068572,"Snap Medical Industries, LLC",Released
1,Jan 12 2023 2:52PM,254949,10,SNAP34112594,"Snap Medical Industries, LLC",Released
2,Jan 12 2023 2:52PM,254949,10,SNAP34125156,"Snap Medical Industries, LLC",Released
3,Jan 12 2023 2:52PM,254949,10,SNAPA1961394,"Snap Medical Industries, LLC",Released
4,Jan 12 2023 2:31PM,254948,10,CTF0011593,"Citieffe, Inc.",Released
5,Jan 12 2023 2:20PM,254947,10,0086296004,ISDIN Corporation,Released
6,Jan 12 2023 2:20PM,254947,10,0086295999,ISDIN Corporation,Released
7,Jan 12 2023 2:20PM,254947,10,0086296000,ISDIN Corporation,Released
8,Jan 12 2023 2:20PM,254947,10,0086296003,ISDIN Corporation,Released
9,Jan 12 2023 2:20PM,254947,10,0086296002,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,254945,Released,3
25,254946,Released,4
26,254947,Released,10
27,254948,Released,1
28,254949,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254945,NaN,NaN,3.0
254946,NaN,NaN,4.0
254947,NaN,NaN,10.0
254948,NaN,NaN,1.0
254949,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254848,0.0,0.0,1.0
254851,0.0,0.0,1.0
254864,30.0,11.0,5.0
254866,0.0,0.0,1.0
254867,8.0,0.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254848,0,0,1
254851,0,0,1
254864,30,11,5
254866,0,0,1
254867,8,0,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254848,0,0,1
1,254851,0,0,1
2,254864,30,11,5
3,254866,0,0,1
4,254867,8,0,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254848,,,1
1,254851,,,1
2,254864,30,11,5
3,254866,,,1
4,254867,8,,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 12 2023 2:52PM,254949,10,"Snap Medical Industries, LLC"
4,Jan 12 2023 2:31PM,254948,10,"Citieffe, Inc."
5,Jan 12 2023 2:20PM,254947,10,ISDIN Corporation
15,Jan 12 2023 2:19PM,254946,10,"Methapharm, Inc."
19,Jan 12 2023 2:10PM,254945,10,Emerginnova
22,Jan 12 2023 1:49PM,254944,19,"GUSA Granules USA, Inc."
23,Jan 12 2023 1:15PM,254938,10,ISDIN Corporation
31,Jan 12 2023 1:07PM,254932,10,ISDIN Corporation
33,Jan 12 2023 12:35PM,254922,21,"NBTY Global, Inc."
34,Jan 12 2023 12:31PM,254921,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 12 2023 2:52PM,254949,10,"Snap Medical Industries, LLC",,,4
1,Jan 12 2023 2:31PM,254948,10,"Citieffe, Inc.",,,1
2,Jan 12 2023 2:20PM,254947,10,ISDIN Corporation,,,10
3,Jan 12 2023 2:19PM,254946,10,"Methapharm, Inc.",,,4
4,Jan 12 2023 2:10PM,254945,10,Emerginnova,,,3
5,Jan 12 2023 1:49PM,254944,19,"GUSA Granules USA, Inc.",,,1
6,Jan 12 2023 1:15PM,254938,10,ISDIN Corporation,,,8
7,Jan 12 2023 1:07PM,254932,10,ISDIN Corporation,,,2
8,Jan 12 2023 12:35PM,254922,21,"NBTY Global, Inc.",,,1
9,Jan 12 2023 12:31PM,254921,21,"NBTY Global, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 2:52PM,254949,10,"Snap Medical Industries, LLC",4,,
1,Jan 12 2023 2:31PM,254948,10,"Citieffe, Inc.",1,,
2,Jan 12 2023 2:20PM,254947,10,ISDIN Corporation,10,,
3,Jan 12 2023 2:19PM,254946,10,"Methapharm, Inc.",4,,
4,Jan 12 2023 2:10PM,254945,10,Emerginnova,3,,
5,Jan 12 2023 1:49PM,254944,19,"GUSA Granules USA, Inc.",1,,
6,Jan 12 2023 1:15PM,254938,10,ISDIN Corporation,8,,
7,Jan 12 2023 1:07PM,254932,10,ISDIN Corporation,2,,
8,Jan 12 2023 12:35PM,254922,21,"NBTY Global, Inc.",1,,
9,Jan 12 2023 12:31PM,254921,21,"NBTY Global, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 2:52PM,254949,10,"Snap Medical Industries, LLC",4,,
1,Jan 12 2023 2:31PM,254948,10,"Citieffe, Inc.",1,,
2,Jan 12 2023 2:20PM,254947,10,ISDIN Corporation,10,,
3,Jan 12 2023 2:19PM,254946,10,"Methapharm, Inc.",4,,
4,Jan 12 2023 2:10PM,254945,10,Emerginnova,3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 2:52PM,254949,10,"Snap Medical Industries, LLC",4.0,NaN,NaN
1,Jan 12 2023 2:31PM,254948,10,"Citieffe, Inc.",1.0,NaN,NaN
2,Jan 12 2023 2:20PM,254947,10,ISDIN Corporation,10.0,NaN,NaN
3,Jan 12 2023 2:19PM,254946,10,"Methapharm, Inc.",4.0,NaN,NaN
4,Jan 12 2023 2:10PM,254945,10,Emerginnova,3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 12 2023 2:52PM,254949,10,"Snap Medical Industries, LLC",4.0,0.0,0.0
1,Jan 12 2023 2:31PM,254948,10,"Citieffe, Inc.",1.0,0.0,0.0
2,Jan 12 2023 2:20PM,254947,10,ISDIN Corporation,10.0,0.0,0.0
3,Jan 12 2023 2:19PM,254946,10,"Methapharm, Inc.",4.0,0.0,0.0
4,Jan 12 2023 2:10PM,254945,10,Emerginnova,3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2549325,55.0,8.0,0.0
15,254866,1.0,0.0,0.0
16,254874,1.0,0.0,0.0
17,254899,1.0,0.0,0.0
19,1274488,17.0,12.0,38.0
21,1274460,4.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2549325,55.0,8.0,0.0
1,15,254866,1.0,0.0,0.0
2,16,254874,1.0,0.0,0.0
3,17,254899,1.0,0.0,0.0
4,19,1274488,17.0,12.0,38.0
5,21,1274460,4.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,55.0,8.0,0.0
1,15,1.0,0.0,0.0
2,16,1.0,0.0,0.0
3,17,1.0,0.0,0.0
4,19,17.0,12.0,38.0
5,21,4.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,55.0
1,15,Released,1.0
2,16,Released,1.0
3,17,Released,1.0
4,19,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,19,21
Status,,,,,,
Completed,0.0,0.0,0.0,0.0,38.0,0.0
Executing,8.0,0.0,0.0,0.0,12.0,1.0
Released,55.0,1.0,1.0,1.0,17.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,19,21
0,Completed,0.0,0.0,0.0,0.0,38.0,0.0
1,Executing,8.0,0.0,0.0,0.0,12.0,1.0
2,Released,55.0,1.0,1.0,1.0,17.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,19,21
0,Completed,0.0,0.0,0.0,0.0,38.0,0.0
1,Executing,8.0,0.0,0.0,0.0,12.0,1.0
2,Released,55.0,1.0,1.0,1.0,17.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()